In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [4]:
# Replace 'your_dataset.csv' with the actual file path
data = pd.read_csv("D:\phishingDetection\data\web-page-phishing.csv")

# Show the first few rows of the dataset
print(data.head())

# Check for missing values and basic statistics
print(data.info())
print(data.describe())


<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
C:\Users\vedik\AppData\Local\Temp\ipykernel_23076\1544790637.py:2: SyntaxWarning: invalid escape sequence '\p'
  data = pd.read_csv("D:\phishingDetection\data\web-page-phishing.csv")


   url_length  n_dots  n_hypens  n_underline  n_slash  n_questionmark  \
0          37       3         0            0        0               0   
1          77       1         0            0        0               0   
2         126       4         1            2        0               1   
3          18       2         0            0        0               0   
4          55       2         2            0        0               0   

   n_equal  n_at  n_and  n_exclamation  n_space  n_tilde  n_comma  n_plus  \
0        0     0      0              0        0        0        0       0   
1        0     0      0              0        0        0        0       0   
2        3     0      2              0        0        0        0       0   
3        0     0      0              0        0        0        0       0   
4        0     0      0              0        0        0        0       0   

   n_asterisk  n_hastag  n_dollar  n_percent  n_redirection  phishing  
0           0         0   

In [6]:
data['phishing'].value_counts()


phishing
0    63715
1    36362
Name: count, dtype: int64

In [8]:
# seperrating the data for analysis
legit = data[data.phishing==0]
fraud = data[data.phishing==1]

In [10]:
print(legit.shape)
print(fraud.shape)


(63715, 20)
(36362, 20)


In [12]:
# compare values for both transactions
data.groupby('phishing').mean()



,url_length,n_dots,n_hypens,n_underline,n_slash,n_questionmark,n_equal,n_at,n_and,n_exclamation,n_space,n_tilde,n_comma,n_plus,n_asterisk,n_hastag,n_dollar,n_percent,n_redirection
phishing,,,,,,,,,,,,,,,,,,,
0,23.589971,2.051950,0.259123,0.045672,0.290732,0.003217,0.026980,0.000031,0.012760,0.000785,0.003265,0.000675,0.000785,0.001946,0.000000,0.000000,0.000000,0.075838,0.391305
1,66.491117,2.526566,0.661130,0.299021,2.615423,0.061493,0.546752,0.060888,0.372119,0.005803,0.007700,0.008773,0.005170,0.003383,0.011276,0.001238,0.005225,0.167895,0.309361


In [14]:
legit_sample = legit.sample(n=36362)

In [18]:
new_dataset =pd.concat([legit_sample,fraud],axis=0)

In [20]:
new_dataset.head()


,url_length,n_dots,n_hypens,n_underline,n_slash,n_questionmark,n_equal,n_at,n_and,n_exclamation,n_space,n_tilde,n_comma,n_plus,n_asterisk,n_hastag,n_dollar,n_percent,n_redirection,phishing
5723,42,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
92584,22,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0
90216,14,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
65209,11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0
79582,12,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [22]:
new_dataset['phishing'].value_counts()


phishing
0    36362
1    36362
Name: count, dtype: int64

In [24]:
new_dataset.groupby('phishing').mean()


,url_length,n_dots,n_hypens,n_underline,n_slash,n_questionmark,n_equal,n_at,n_and,n_exclamation,n_space,n_tilde,n_comma,n_plus,n_asterisk,n_hastag,n_dollar,n_percent,n_redirection
phishing,,,,,,,,,,,,,,,,,,,
0,23.633464,2.051180,0.260244,0.049035,0.292146,0.002970,0.026676,0.000000,0.012678,0.000715,0.003438,0.000715,0.000963,0.001430,0.000000,0.000000,0.000000,0.076921,0.392030
1,66.491117,2.526566,0.661130,0.299021,2.615423,0.061493,0.546752,0.060888,0.372119,0.005803,0.007700,0.008773,0.005170,0.003383,0.011276,0.001238,0.005225,0.167895,0.309361


In [30]:
X= new_dataset.drop(columns='phishing' ,axis=1)
Y = new_dataset['phishing']


In [32]:
print(X)


        url_length  n_dots  n_hypens  n_underline  n_slash  n_questionmark  \
5723            42       2         0            0        0               0   
92584           22       2         0            0        0               0   
90216           14       2         0            0        0               0   
65209           11       1         0            0        0               0   
79582           12       2         0            0        0               0   
...            ...     ...       ...          ...      ...             ...   
100065          78       2         1            0        3               0   
100067          41       3         1            0        2               0   
100068          41       2         1            0        4               0   
100074          70       2         1            0        5               0   
100075          28       2         0            0        1               0   

        n_equal  n_at  n_and  n_exclamation  n_space  n_tilde  

In [34]:
print(Y)


5723      0
92584     0
90216     0
65209     0
79582     0
         ..
100065    1
100067    1
100068    1
100074    1
100075    1
Name: phishing, Length: 72724, dtype: int64


In [44]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size= 0.2, stratify=Y, random_state=2)

In [46]:
print(X.shape,X_train.shape,X_test.shape)

(72724, 19) (58179, 19) (14545, 19)


In [62]:
model =XGBClassifier()

In [95]:
#training the xgbclassifier model with training data
model = XGBClassifier(objective="binary:logistic", eval_metric="logloss")
model.fit(X_train, Y_train)
model.save_model("phishing_detection_model.json")

In [68]:
# Predictions on training data
train_preds = model.predict(X_train)

In [72]:
train_accuracy = accuracy_score(Y_train, train_preds)
print(f"Training Accuracy: {train_accuracy:.2f}")

Training Accuracy: 0.90


In [76]:
#  Evaluate on test data
test_preds = model.predict(X_test)
test_accuracy = accuracy_score(Y_test, test_preds)
print(f"Test Accuracy: {test_accuracy:.2f}")

Test Accuracy: 0.89


In [97]:
# Check if the model is fitted
if model._Booster is not None:
    print("Model is trained!")
else:
    print("Model is not trained.")



Model is trained!
